# Evaluate SR output - hallucations and faithfullness

In [1]:
import sys

sys.path = ['/Users/jonathan/Projects/semantic-retrieval/python/src'] + sys.path

## Reproduce reports

In [2]:
# Reproduce report 
# in semantic-retrieval/ dir. NOTE: namespace is for pinecone.
 python python/src/semantic_retrieval/examples/financial_report/generate_report.py \
    --log-level=info \
    --client-name=my_client --viewer-role="advisor/my_name" \
    --namespace='my_namespace' \
    --sample-output-path=my_report.out \ 
    --variant-name='covid' # or 'net_cash_flow'

## Example eval input: 
* Client portfolio database (local CSV)
* Intermediate SR results: raw retrieved document chunks
* SR final output saved in local files

In [3]:
ROOT_DATA_DIR = "../../../../../examples/example_data/financial_report/"

print("Ready")

Ready


In [12]:
from semantic_retrieval.evaluation import metrics
from semantic_retrieval.examples.financial_report.lib.eval import (
    get_test_suite,
    test_case_to_sample_eval_params,
)

evaluation_params_list = [
    await test_case_to_sample_eval_params(tc, ROOT_DATA_DIR) for tc in get_test_suite()
]

# Separate out the valid test params from the error ones.
evaluation_params_valid = [ep.unwrap() for ep in evaluation_params_list if ep.is_ok()]

print("Evaluating the following test cases:")
for eval_param in evaluation_params_valid:
    print(f"\n\nEval params: {eval_param}")


Evaluating the following test cases:


Eval params: 
SampleEvaluationParams:
	self.output_sample={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x109c3efc0>, gt={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}, name='net_income_vs_retrieved')


Eval params: 
SampleEvaluationParams:
	self.output_sample={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x109c3efc0>, gt={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}, name='covid_vs_retrieved')


Eval params: 
SampleEvaluationParams:
	self.output_sample={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x109c3efc0>, gt={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA'}, name='net_income_e2e')


Eval params: 
SampleEvaluationParams:
	self.output_sample={'AMZN', 'JNJ', 'MRK', 'AAPL', 'UNH', 'PEP', 'NVDA

In [13]:
from semantic_retrieval.evaluation.lib import (
    eval_res_to_df,
    evaluate,
)

eval_res = evaluate(evaluation_params_valid)

if eval_res.is_err():
    logger.critical(f"Error evaluating: {eval_res.err()}")

df_eval_res = eval_res_to_df(eval_res.unwrap())

display(df_eval_res.set_index(["name", "interpretation"]))

,,value
name,interpretation,
net_income_vs_retrieved,Jaccard similarity with ground truth,1.0
covid_vs_retrieved,Jaccard similarity with ground truth,1.0
net_income_e2e,Jaccard similarity with ground truth,1.0
covid_e2e,Jaccard similarity with ground truth,1.0
